In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp2d

In [ ]:
datasetIncendios = pd.read_csv("./modis_2021_Spain_incendios_with_clima.csv")
datasetIncendios = datasetIncendios.query('latitude > 30')

datasetIncendios["Temperatura máxima (ºC)"] =  np.zeros(len(datasetIncendios))
datasetIncendios["Temperatura mínima (ºC)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Temperatura media (ºC)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Racha (km/h)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Velocidad máxima (km/h)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Precipitación 00-24h (mm)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Precipitación 00-06h (mm)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Precipitación 06-12h (mm)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Precipitación 12-18h (mm)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Precipitación 18-24h (mm)"] = np.zeros(len(datasetIncendios))
datasetIncendios.to_csv("./modis_2021_Spain_incendios_with_clima.csv", index=False)

In [ ]:
columns = ['Temperatura máxima (ºC)',
'Temperatura mínima (ºC)',
'Temperatura media (ºC)',
'Racha (km/h)',
'Velocidad máxima (km/h)',
'Precipitación 00-24h (mm)',
'Precipitación 00-06h (mm)',
'Precipitación 06-12h (mm)',
'Precipitación 12-18h (mm)',
'Precipitación 18-24h (mm)']
datasetIncendios = pd.read_csv("./modis_2021_Spain_incendios_with_clima.csv")

def getClosestValue(index, colum):
    columnIndexdate = datasetIncendios.columns.get_loc("acq_date")
    print(datasetIncendios.iloc[index,columnIndexdate])
    df = pd.read_csv(f'./ClimaDataSmall/Aemet{datasetIncendios.iloc[index,columnIndexdate]}.csv')
    columnIndex = df.columns.get_loc(colum)
    df = df.iloc[:,[-1,-2,columnIndex]]
    df = df.query('Latitude > 30 and Latitude < 44 and Longitude > -20')
    if str(df.iloc[1,2]).find(' ') != -1:
        for _ in range(len(df)):
            df.iloc[_,2] = float(df.iloc[_,2][:df.iloc[_,2].find(' ')])
    a = []
    for _ in df.iloc[:, 2].values:
        a.append(float(_))

    df2 = df.drop([colum], 1)
    df3 = df[colum]
    lat = df2['Latitude']
    lon = df2['Longitude']
    values = np.array(df3,dtype='float64')
    arraylat = np.array(lat,dtype='float64')
    arraylon = np.array(lon,dtype='float64')
    i = interp2d(arraylat,arraylon , values)
    columnIndex = datasetIncendios.columns.get_loc(colum)
    est_u = i(datasetIncendios.iloc[index,0],datasetIncendios.iloc[index,1])
    print(datasetIncendios.iloc[index,0],datasetIncendios.iloc[index,1])
    datasetIncendios.iloc[index,columnIndex] = round(est_u[0], 6)
    print(f"{colum} : {est_u[0]}" )


for index in range(len(datasetIncendios)):
    print(f"{index} of : {len(datasetIncendios)}")
    for colum in columns:
        getClosestValue(index, colum)

datasetIncendios.to_csv("./modis_2021_Spain_incendios_with_clima2.csv", index=False)

In [16]:

datasetIncendios = pd.read_csv("./modis_2021_Spain_incendios_with_clima2.csv")
datasetIncendios

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,...,Temperatura máxima (ºC),Temperatura mínima (ºC),Temperatura media (ºC),Racha (km/h),Velocidad máxima (km/h),Precipitación 00-24h (mm),Precipitación 00-06h (mm),Precipitación 06-12h (mm),Precipitación 12-18h (mm),Precipitación 18-24h (mm)
0,37.0886,-5.8802,300.4,1.1,1.0,2021-01-02,1125,Terra,MODIS,37,...,21.353232,-2.533881e+00,9.298166,22.578122,1.503172e+01,0.015306,-0.002044,0.015833,0.000455,0.000000
1,36.6599,-4.8777,305.5,1.8,1.3,2021-01-02,1304,Aqua,MODIS,62,...,11.095920,5.196013e+00,8.882916,34.208437,1.718057e+01,0.015459,-0.003381,0.012758,0.009419,0.000000
2,36.8729,-2.4486,304.8,1.2,1.1,2021-01-02,2228,Terra,MODIS,62,...,14.367423,2.501103e+04,10.728333,45.369113,-1.031346e+01,0.047007,-0.000073,0.025924,0.026062,-0.000000
3,36.8745,-2.4347,334.7,1.2,1.1,2021-01-02,2228,Terra,MODIS,100,...,14.432656,1.176415e+04,10.879741,46.400042,-1.160703e+01,0.040432,-0.000076,0.027308,0.027086,-0.000000
4,40.2352,-0.8699,320.1,1.1,1.1,2021-01-04,1112,Terra,MODIS,80,...,1.655655,-2.984083e+00,2.262419,-10229.156936,1.321558e+02,0.500131,-0.078985,0.646705,-0.000356,0.157702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2903,43.2687,-5.2530,307.4,1.1,1.0,2021-12-31,2206,Terra,MODIS,71,...,-58353.047882,2.400191e+05,28.610231,9.377339,-3.601731e+02,944.601218,0.000000,0.003399,0.000000,0.000000
2904,43.1755,-5.4329,309.2,1.1,1.0,2021-12-31,2206,Terra,MODIS,77,...,-11254.205210,1.132960e+05,30.303386,-25.230159,-3.039460e+02,1154.378719,0.000000,0.002718,0.000000,0.000000
2905,43.2442,-3.7813,316.1,1.0,1.0,2021-12-31,2206,Terra,MODIS,93,...,-12152.917570,2.278537e+07,66.201452,36.879534,-3.459886e+06,0.529058,-0.000000,0.000000,-0.000000,-0.003962
2906,43.1631,-5.5095,303.0,1.1,1.0,2021-12-31,2206,Terra,MODIS,47,...,22.201604,8.233332e+04,27.884531,-46.529262,-2.409407e+02,1223.699761,0.000000,0.001842,0.000000,0.000000
